### Imports

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys

In [1]:
import numpy as np
import pandas as pd

In [34]:
pd.set_option('display.max_columns', 500)

In [11]:
import json

In [12]:
from pandas.io.json import json_normalize

In [6]:
sys.path.append('../src/')

In [13]:
from data_utils import load_file

### Load data 

In [14]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [15]:
train_df = load_file('../data/train.csv',
                     nrows=10000,
                     converters={column: json.loads for column in JSON_COLUMNS})

In [26]:
%%time
for column in JSON_COLUMNS:
    tmp_df = json_normalize(train_df[column])
    tmp_df.columns = [f'{train_df[column].name}_{name}' for name in tmp_df.columns]
    train_df = train_df.drop(column, axis=1).merge(tmp_df, left_index=True, right_index=True)

CPU times: user 2.03 s, sys: 243 ms, total: 2.28 s
Wall time: 2.57 s


In [38]:
def flatten_jsons_in_df(path_to_df, nrows=None):
    df = load_file(path_to_df,
                   nrows=nrows,
                   converters={column: json.loads for column in JSON_COLUMNS})
    for column in JSON_COLUMNS:
        tmp_df = json_normalize(df[column])
        tmp_df.columns = [
            f'{df[column].name}_{name}' for name in tmp_df.columns]
        df = df.drop(column, axis=1).merge(
            tmp_df, left_index=True, right_index=True)
        
    return df

In [39]:
train_df = flatten_jsons_in_df('../data/train.csv',
                               nrows=10000)

In [41]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
channelGrouping                                      10000 non-null object
date                                                 10000 non-null int64
fullVisitorId                                        10000 non-null object
sessionId                                            10000 non-null object
socialEngagementType                                 10000 non-null object
visitId                                              10000 non-null int64
visitNumber                                          10000 non-null int64
visitStartTime                                       10000 non-null int64
device_browser                                       10000 non-null object
device_browserSize                                   10000 non-null object
device_browserVersion                                10000 non-null object
device_deviceCategory                                10000 non-null obj

In [42]:
test_df = flatten_jsons_in_df('../data/test.csv',
                               nrows=10000)

In [43]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 53 columns):
channelGrouping                                      10000 non-null object
date                                                 10000 non-null int64
fullVisitorId                                        10000 non-null object
sessionId                                            10000 non-null object
socialEngagementType                                 10000 non-null object
visitId                                              10000 non-null int64
visitNumber                                          10000 non-null int64
visitStartTime                                       10000 non-null int64
device_browser                                       10000 non-null object
device_browserSize                                   10000 non-null object
device_browserVersion                                10000 non-null object
device_deviceCategory                                10000 non-null obj

In [44]:
test_cols = set(test_df.columns)
train_cols = set(train_df.columns)
train_cols - test_cols

{'totals_transactionRevenue'}